In [1]:
import pandas as pd


<h4>Read FIN001 sample report and paycheck table</h4>

In [2]:
FIN001filepath = r"C:\Users\awidjaja\Documents\RFL- POC\Input_Files\UC_FIN001_PAYCHECK_DEDUCTIONS_774084231.xls"
df = pd.read_excel(FIN001filepath, header=1)
paycheckfilepath=r"C:\Users\awidjaja\Documents\RFL- POC\Input_Files\PYCK DATES_V2.xlsx"
dfPaycheck = pd.read_excel(paycheckfilepath,sheet_name='cleaned')

In [ ]:
df['Payback Amount'][0] = 3
df['Refund Amount'][10] = 33.5
df['Not Taken'][11] = 270
df['Not Taken'][12] = -250
df['Reason'][12] = "Some random reasons..."
df['Paycheck Status'][13] = "NOT CONFIRMED"
df['Paycheck Option'][13] = "Not Advice"
df['Off Cycle'][20] ='Y'

<h4>Highlight variances on Payback Amount, Refund Amount, Not Taken, Reason , Paycheck Status , Paycheck Option, OffCycle</h4>
<p>Conditions are as follow:</p>
<ul>
    <li>if Payback Amount (> or <) 0.00 then color cell </li>
    <li>if Refund Amount (> or <) 0.00, then color cell </li>
    <li>if Not Taken (> or <) 0.00, then color cell </li>
    <li>if Reason != null, then color cell </li>
    <li>if Paycheck Status != 'Confirmed', then color cell </li>
    <li>if Paycheck Options != 'Advice', then color cell </li>
    <li>if Off Cycle != 'N'</li>
</ul>

In [5]:
df.fillna("Null",inplace=True)

<p>Note about Styler object:</p>
<ol>
<li>When you call df.style, it returns a pandas.io.formats.style.Styler object, not a DataFrame.</li>
<li>The Styler object provides methods for applying various styles (e.g., highlighting, formatting, coloring) to the DataFrame's display.</li>
<li>It is not intended for data selection or modification, which are operations performed on the underlying DataFrame itself.</li>
</ol>
    <p>Solution: To resolve this error, ensure that you are performing data selection or manipulation on the original DataFrame before applying styling, or access the underlying DataFrame data from the Styler object if necessary.</p>
    <p>In summary, the Styler object is for presentation and not for data manipulation using subscripting. Always ensure you are interacting with the appropriate object (DataFrame for data, Styler for styling) for the intended operation.</p>


In [6]:
def highlightAnomalies(val, col_name):
    if col_name == 'Payback Amount' and (val >0 or val <0):
        return 'background-color: pink'
    elif col_name =='Refund Amount' and (val > 0 or val<0):
        return 'background-color:pink'
    elif col_name =='Not Taken' and (val > 0 or val<0):
        return 'background-color:pink'
    elif col_name == 'Reason' and val != "Null":
        return 'background-color:pink'
    elif col_name =='Paycheck Status' and val !="Confirmed":
        return 'background-color:pink'
    elif col_name =='Paycheck Option' and val !="Advice":
        return 'background-color:pink'
    elif col_name =='Off Cycle' and val !="N":
        return 'background-color:pink'
    else:
        return ''
    

dfStyler = df.style.applymap(lambda x: highlightAnomalies(x, 'Payback Amount'), subset=['Payback Amount'])\
.applymap(lambda x: highlightAnomalies(x, 'Refund Amount'), subset=['Refund Amount'])\
.applymap(lambda x: highlightAnomalies(x, 'Not Taken'), subset=['Not Taken'])\
.applymap(lambda x: highlightAnomalies(x, 'Reason'), subset=['Reason'])\
.applymap(lambda x: highlightAnomalies(x, 'Paycheck Status'), subset=['Paycheck Status'])\
.applymap(lambda x: highlightAnomalies(x, 'Paycheck Option'), subset=['Paycheck Option'])\
.applymap(lambda x: highlightAnomalies(x, 'Off Cycle'), subset=['Off Cycle'])




In [ ]:
dfStyler

<h4>Split dataframe by plan type</h4>

In [3]:
plantype_dfs={}

In [4]:
for plan, planDF in df.groupby(by=['Plan Type']):
    plantype_dfs[plan] = planDF

In [5]:
plantype_dfs.keys()

dict_keys(['403(b)', 'Basic Disability', 'Dental', 'Employee & Dependent AD&D', 'Exp Dependent Life - Spouse/DP', 'Health Savings Account', 'Legal Insurance', 'Life', 'Medical', 'Supplemental Life', 'UC Retirement Plan', 'Vision', 'Voluntary LongTerm Disability', 'Voluntary ShortTerm Disability'])

In [36]:
plantype_dfs['Medical']['Deduction Classification'].unique()

array(['Nontaxable Benefit', 'Before-Tax'], dtype=object)

In [60]:
plantype_dfs['Dental']['Deduction Classification'].unique()

array(['Nontaxable Benefit'], dtype=object)

In [61]:
plantype_dfs['Vision']['Deduction Classification'].unique()

array(['Nontaxable Benefit'], dtype=object)

In [47]:
plantype_dfs['Medical'][['Paycheck Issue Date','Current Deduction','Deduction Classification']].sort_values(by='Deduction Classification')

,Paycheck Issue Date,Current Deduction,Deduction Classification
8,2024-04-24,218.16,Before-Tax
24,2024-05-08,218.16,Before-Tax
39,2024-05-22,545.40,Before-Tax
128,2024-08-28,218.16,Before-Tax
56,2024-06-05,545.40,Before-Tax
68,2024-06-18,218.16,Before-Tax
143,2024-09-11,218.16,Before-Tax
84,2024-07-03,218.16,Before-Tax
114,2024-08-14,218.16,Before-Tax
97,2024-07-17,218.16,Before-Tax


<h4>Analyze  year over year change in average rate in Medical plan</h4>

In [48]:
plantype_dfs['Medical']['Year'] = plantype_dfs['Medical']['Paycheck Issue Date'].dt.year 

In [55]:
yearly_summary = plantype_dfs['Medical'].groupby(['Year','Deduction Classification'] )['Current Deduction'].mean().reset_index()
yearly_summary
#yearly_summary.rename(columns={'Current Deduction': 'Average Rate'}, inplace=True)
#yearly_summary['Change Rate Max'] = yearly_summary['Average Rate'].max()
#print(yearly_summary)

,Year,Deduction Classification,Current Deduction
0,2024,Before-Tax,283.608000
1,2024,Nontaxable Benefit,727.917273


<h5>highlighting by count</h5>

In [ ]:
rate_count_grpby = plantype_dfs['Medical'].groupby(by=['Current Deduction']).count()
#rate_count_grpby
def highlight_by_count(val, threshold_variance):
    if(rate_count_grpby['Employee ID'].loc[val] <=2):
        return 'background-color: red'
    return ''

styled_df = plantype_dfs['Medical'].style.applymap(lambda x: highlight_by_count(x, rate_count_grpby), subset=['Current Deduction'])
styled_df

In [11]:
plantype_dfs['403(b)']['Paycheck Issue Date']

14    2024-05-08
31    2024-05-22
47    2024-06-05
63    2024-06-18
75    2024-07-03
91    2024-07-17
103   2024-07-31
106   2024-08-14
122   2024-08-28
134   2024-09-11
Name: Paycheck Issue Date, dtype: datetime64[ns]

In [12]:
plantype_dfs['Basic Disability']['Paycheck Issue Date']

15    2024-05-08
32    2024-05-22
48    2024-06-05
76    2024-07-03
107   2024-08-14
135   2024-09-11
Name: Paycheck Issue Date, dtype: datetime64[ns]

<h4>Identifying missed paycheck dates </h4>

In [13]:
minDate = plantype_dfs['403(b)']['Paycheck Issue Date'].min().strftime("%Y-%m-%d")
maxDate = plantype_dfs['403(b)']['Paycheck Issue Date'].max().strftime("%Y-%m-%d")
minYear = minDate.split("-")[0]
maxYear = maxDate.split("-")[0]
print(minDate)
print(minYear)
print(maxDate)
print(maxYear)

2024-05-08
2024
2024-09-11
2024


In [53]:
dfPaycheck[2024][(dfPaycheck[2024]>= minDate)& (dfPaycheck[2024] <=maxDate)]
PaycheckDateList=list(dfPaycheck[2024][(dfPaycheck[2024]>= minDate)& (dfPaycheck[2024] <=maxDate)].dt.strftime("%Y-%m-%d"))
PaycheckDateList.append('2024-06-20')
PaycheckDateList.append('2024-06-21')
PaycheckDateList

['2024-05-08',
 '2024-05-22',
 '2024-06-05',
 '2024-06-19',
 '2024-07-03',
 '2024-07-17',
 '2024-08-14',
 '2024-08-28',
 '2024-09-11',
 '2024-06-20',
 '2024-06-21']

In [49]:
#plantype_dfs['403(b)']['Paycheck Issue Date']
PaycheckIssueDateList = list(plantype_dfs['403(b)']['Paycheck Issue Date'].dt.strftime("%Y-%m-%d"))
PaycheckIssueDateList

['2024-05-08',
 '2024-05-22',
 '2024-06-05',
 '2024-06-18',
 '2024-07-03',
 '2024-07-17',
 '2024-07-31',
 '2024-08-14',
 '2024-08-28',
 '2024-09-11']

In [54]:
xx = list(set(PaycheckDateList) - set(PaycheckIssueDateList))
print(xx)

['2024-06-21', '2024-06-20', '2024-06-19']


In [23]:
plantype_dfs['403(b)'].columns

Index(['Business Unit', 'Employee ID', 'Employee Record', 'Employee Name',
       'Pay Group', 'Pay Period End Date', 'Paycheck Issue Date',
       'Employee Type', 'Plan Type', 'Benefit Plan', 'Deduction Code',
       'Deduction Code Descr', 'Deduction Classification', 'Current Deduction',
       'Payback Amount', 'Refund Amount', 'Not Taken', 'Reason',
       'Calculated Base', 'Paycheck Status', 'Paycheck Option', 'Off Cycle'],
      dtype='object')

In [60]:
selected_columns=['Business Unit', 'Employee ID', 'Employee Record', 'Employee Name',
       'Pay Group', 'Pay Period End Date',
       'Employee Type', 'Plan Type', 'Benefit Plan']
lastindex = plantype_dfs['403(b)'].index[len(plantype_dfs['403(b)'])-1]
copiedRowLastIndex = plantype_dfs['403(b)'].loc[lastindex, selected_columns]

newrow = pd.Series({col: copiedRowLastIndex.get(col,pd.NA) for col in plantype_dfs['403(b)'].columns})
newrow['Paycheck Issue Date'] = xx[0]

print(newrow.dtype)



object


In [ ]:

"""
# Original DataFrame
df123 = pd.DataFrame({
    "A": [1, 2],
    "B": [3, 4],
    "C": [5, 6],
    "D": [7, 8]
})

# Select row index 0 and only some columns
selected_columns = ["A", "C"]
partial_row = df123.loc[0, selected_columns]

# Create a new row with NaNs for missing columns
new_row = pd.Series({col: partial_row.get(col, pd.NA) for col in df123.columns})

# Append the new row
df123 = pd.concat([df123, pd.DataFrame([new_row])], ignore_index=True)

print(df123)
"""